In [14]:
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoProcessor

import requests

from PIL import Image

In [45]:
model = VisionEncoderDecoderModel.from_pretrained("showpiece/donut4cover_of_books")
processor = AutoProcessor.from_pretrained("showpiece/donut4cover_of_books")

In [51]:
raw_image = Image.open('test_ru.png').convert('RGB')
raw_image.show()

In [53]:
inputs = processor(raw_image, return_tensors="pt")

In [54]:
out = model.generate(**inputs)

In [55]:
processor.decode(out[0])

'<s_cord-v2> Pok3 Ca50 (Kintai Manga) (Volume 1) (Volume 1) (Volume 1) (Volume 1) (Volume 1) (Volume 1) (Volume 1) (Volume 4) (Volume 4) (Volume 4) (Volume 4) (Volume 4) (Volume 4) (Volume 4) (Volume 4) (Volume 4) (Volume 4) (Volume 4) (Volume 4) (Volume 4) (Volume 4) (Volume 4) (Volume 4) (Volume 4))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))) Poy3 Ca50,000,000,000,000,000,000</s>'

In [44]:
# здесь была английская обложка
processor.decode(out[0])

"<s_cord-v2> Harry Potter and the Sorcerer's Stone (The Sorcerer's Stone) (Volume 1) (Volume 1) (Volume 1) (Volume 1) (Volume 1) (Volume 1) (Volume 1) (Volume 1) (Volume 1) (Volume 1) (Volume 1) (Volume 1) (Volume 1) (Volume 1))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))) Porter, Porter, Porter, Porter, Porter, Porter, Porter, Porter, Porter, Porter, Porter, Porter, Porter, Porter, Porter,</s>"

In [1]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-giant')
model = AutoModel.from_pretrained('facebook/dinov2-giant')

preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

C:\Users\Nikita\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nikita\.cache\huggingface\hub\models--facebook--dinov2-giant. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

In [34]:
image1 = Image.open('test_harry_1.jpg').convert('RGB')
image2 = Image.open('test_harry_2.jpg').convert('RGB')
image3 = Image.open('test_harry_3.jpg').convert('RGB')

In [35]:
def embed_picture(image):
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.detach().numpy()[0].flatten()
    return  embedding

In [36]:
h1 = embed_picture(image1)
h2 = embed_picture(image2)
h3 = embed_picture(image3)

In [45]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a, b):
    return dot(a, b)/(norm(a)*norm(b))

In [39]:
cos_sim(h1, h2)

0.31547517

In [ ]:
взять баундинг боксы 

In [40]:
cos_sim(h1, h3)

0.17002004

In [43]:
h4 = embed_picture(Image.open('test_ru.png').convert('RGB'))

In [50]:
cos_sim(h1, h4)

0.16024242

In [22]:
h1.flatten()

array([ 0.9686465 , -1.0845854 ,  1.1518844 , ..., -1.2883371 ,
        1.4850776 ,  0.18848781], dtype=float32)